In [3]:

#import libraries
#from pyspark.sql.functions import avg
import matplotlib.pyplot as plt
#plt.rcParams['font.size'] = 22
import warnings    # To suppress warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
np.set_printoptions(threshold=sys.maxsize)


In [6]:


df1 = pd.read_csv('PRED_31MAR17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df2 = pd.read_csv('PRED_30JUN17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df3 = pd.read_csv('PRED_31OCT17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df4 = pd.read_csv('PRED_31DEC17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df5 = pd.read_csv('PRED_31MAR18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df6 = pd.read_csv('PRED_30JUN18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df7 = pd.read_csv('PRED_31OCT18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df8 = pd.read_csv('PRED_31DEC18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df9 = pd.read_csv('PRED_MARR19.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])


list_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9]
for i in list_df:
  i = i.drop_duplicates().reset_index(drop = True)


In [7]:

quarter_dict = {3: 1, 6: 2, 10: 3, 12: 4} # quarter dict
for i in list_df:
    i['Year'] = i[i['EMP_STATUS'] == 'I']['HIRE_EXIT_DATE'].max().year
    i['Quarter'] = pd.Series(i[i['EMP_STATUS'] == 'I']['HIRE_EXIT_DATE'].max().month).apply(quarter_dict.get)[0]
    i['Month'] = i[i['EMP_STATUS'] == 'I']['HIRE_EXIT_DATE'].max().month
for i in list_df:
  i['ID']=i['Quarter'].astype(str)+'_'+i['Year'].astype(str)

In [8]:
training_set = pd.concat([df1,df2,df3,df4,df5,df6,df7]).drop_duplicates().reset_index(drop=True)
validation_set = pd.concat([df8,df9]).drop_duplicates().reset_index(drop=True)


In [9]:
'''
for i in list_df:
  i.replace(' ', np.nan, inplace=True)
  '''

"\nfor i in list_df:\n  i.replace(' ', np.nan, inplace=True)\n  "

In [10]:
temp1=df1[df1.EMP_STATUS=='A']
temp2=df2[df2.EMP_STATUS=='A']
temp3=df3[df3.EMP_STATUS=='A']
temp4=df4[df4.EMP_STATUS=='A']
temp5=df5[df5.EMP_STATUS=='A']
temp6=df6[df6.EMP_STATUS=='A']
temp7=df7[df7.EMP_STATUS=='A']
temp8=df8[df8.EMP_STATUS=='A']
temp9=df9[df9.EMP_STATUS=='A']



In [11]:
training_set.shape

(507392, 41)

In [12]:
training_set.columns

Index(['MASK_EMPLOYEEID', 'EMPLOYEE_AGE', 'GENDER', 'NO_OF_LEAVES',
       'TOTAL_REPORTEES', 'BAND', 'DEPARTMENT_ID', 'NO_OF_MANAGER_REPORTEES',
       'TECHM_EXPERIENCE', 'TOTAL_EXPERIENCE', 'LAST_HIKE_PERCENT',
       'MANAGER_RATING', 'NO_OF_PREVIOUS_EMPLOYERS', 'TENURE_LAST_EMPLOPYER',
       'HIGHEST_EDUCATION', 'QUARTILE', 'LAST_PROMOTION_DATE',
       'TOTAL_PROMOTIONS', 'MANAGER_LAST_PROMOTION', 'LAST_ONSITE_TRAVEL_DATE',
       'LAST_ONSITE_TRAVEL_DAYS', 'REWARDS', 'PRIMARY_SKILL',
       'SECONDARY_SKILL', 'BASE_LOCATION_CITY', 'CURRENT_LOCATION_CITY',
       'HIRE_LOCATION_CITY', 'RESIGNATION_FLG', 'BILLABILITY_STATUS',
       'CURRENT_RATING', 'LAST_RATING', 'ALL_PREVIOUS_RAINGS',
       'LATEST_HIKE_PERCENT', 'OFFICERCODE_DESC', 'GRIEVANCE_FLG',
       'EMP_STATUS', 'HIRE_EXIT_DATE', 'Year', 'Quarter', 'Month', 'ID'],
      dtype='object')

In [ ]:
training_set['EMP_STATUS'].value_counts()

In [ ]:
training_set[training_set['MASK_EMPLOYEEID']=='93578A9A4D51DBF7']

In [ ]:
fd1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],df2,how='inner',on=['MASK_EMPLOYEEID'])
fd2=pd.merge(temp2[['MASK_EMPLOYEEID','EMP_STATUS']],df3,how='inner',on=['MASK_EMPLOYEEID'])
fd3=pd.merge(temp3[['MASK_EMPLOYEEID','EMP_STATUS']],df4,how='inner',on=['MASK_EMPLOYEEID'])
fd4=pd.merge(temp4[['MASK_EMPLOYEEID','EMP_STATUS']],df5,how='inner',on=['MASK_EMPLOYEEID'])
fd5=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],df6,how='inner',on=['MASK_EMPLOYEEID'])
fd6=pd.merge(temp6[['MASK_EMPLOYEEID','EMP_STATUS']],df7,how='inner',on=['MASK_EMPLOYEEID'])
fd7=pd.merge(temp7[['MASK_EMPLOYEEID','EMP_STATUS']],df8,how='inner',on=['MASK_EMPLOYEEID'])
fd8=pd.merge(temp8[['MASK_EMPLOYEEID','EMP_STATUS']],df9,how='inner',on=['MASK_EMPLOYEEID'])

In [ ]:
'''lisst_df = [fd8,fd8,fd8,fd8.fd8,fd8,fd8,fd8]
for i in lisst_df:
  
    i['Leaves_avg'] = i[i['NO_OF_LEAVES'] == 'I']['HIRE_EXIT_DATE'].max().year
    i['Quarter'] = pd.Series(i[i['EMP_STATUS'] == 'I']['HIRE_EXIT_DATE'].max().month).apply(quarter_dict.get)[0]
    i['Month'] = i[i['EMP_STATUS'] == 'I']['HIRE_EXIT_DATE'].max().month'''

In [ ]:
'''df_mar_17_target = df_mar_17_A.merge(df_jun_17_I[['MASK_EMPLOYEEID', 'EMP_STATUS']], on = ['MASK_EMPLOYEEID'], how = 'left')
df_mar_17_target.rename(columns = {'EMP_STATUS_y': 'Target'}, inplace = True)
df_mar_17_target['Target'] = df_mar_17_target['Target'].fillna('A')
'''

In [ ]:



#Deletng EMP_STATUS column from original df
for i in range(8):
    data=globals()["fd" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["fd" + str(i + 1)]=data

In [ ]:
f1 = df2.append(df3)
f1 = df2.append(df4)
triple1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],f1,how='inner',on=['MASK_EMPLOYEEID'])

f2 = df3.append(df4)
f2 = df3.append(df5)
triple2=pd.merge(temp2[['MASK_EMPLOYEEID','EMP_STATUS']],f2,how='inner',on=['MASK_EMPLOYEEID'])

f3 = df4.append(df5)
f3 = df4.append(df6)
triple3=pd.merge(temp3[['MASK_EMPLOYEEID','EMP_STATUS']],f3,how='inner',on=['MASK_EMPLOYEEID'])

f4 = df5.append(df6)
f4 = df5.append(df7)
triple4=pd.merge(temp4[['MASK_EMPLOYEEID','EMP_STATUS']],f4,how='inner',on=['MASK_EMPLOYEEID'])


f5 = df6.append(df7)
f5 = df6.append(df8)
triple5=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],f5,how='inner',on=['MASK_EMPLOYEEID'])


list_f = [triple1,triple2,triple3,triple4,triple5]
for i in list_f:
  i['MASK_EMPLOYEEID'] = i['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)


#triples['MASK_EMPLOYEEID'].drop_duplicates().reset_index(drop = True)
  

In [ ]:

#Deletng EMP_STATUS column from original df
for i in range(5):
    data=globals()["triple" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["triple" + str(i + 1)]=data

In [ ]:
y1 = df2.append(df3)
y1 = df2.append(df4)
year1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],y1,how='inner',on=['MASK_EMPLOYEEID'])
year1['MASK_EMPLOYEEID'] = year1['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)
#year1 = year1.drop_duplicates(keep=False,inplace=True)

y2 = df6.append(df7)
y2 = df6.append(df8)
year2=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],y2,how='inner',on=['MASK_EMPLOYEEID'])
year2['MASK_EMPLOYEEID'] = year2['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)
#year2 = year2.drop_duplicates(keep=False,inplace=True)
'''
'''

In [ ]:

#Deletng EMP_STATUS column from original df
for i in range(2):
    data=globals()["year" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["year" + str(i + 1)]=data
    '''
    '''

#mapping completed

In [ ]:
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
# print(fd1.QUARTILE.value_counts())
for i in llist:
  print('------------------')
  print(i.QUARTILE.value_counts())
  print((i[i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100)/i.QUARTILE.value_counts())

In [ ]:
labels=fd1.QUARTILE.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=(fd1[fd1.TARGET_VALUE=='I'].QUARTILE.value_counts()*100)/fd1.QUARTILE.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("Quartile Counts",color="saddlebrown",fontsize=15)
display()

In [ ]:
labels=fd1.QUARTILE.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("Education Field Counts",color="saddlebrown",fontsize=15)
display()

In [ ]:
var1=pd.crosstab(data["QUARTILE"],data['TARGET_VALUE'])
var1.div(var1.sum(1).astype(float), axis=0).plot(kind="bar", stacked=False, figsize = (10,5)) 
  #plt.show()
display()

In [ ]:
#total percentage of experienced and inexperienced employee in each quartile for each quarter
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print('Quarter ' , str(j))
  print('total percent of inexperienced employee')
  print(i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())
  print('total percent of experienced employee')
  print(i[i.TOTAL_EXPERIENCE>3].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())
  j+=1

In [ ]:
#percentage of experienced and inexperienced people who are attriting


llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print('Quarter ' , str(j))
  print('total percent of inexperienced employee')
  print(i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100/i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts())
  print('total percent of experienced employee')
  print(i[i.TOTAL_EXPERIENCE>3][i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100/i[i.TOTAL_EXPERIENCE>3].QUARTILE.value_counts())
  print('-----------')
  j+=1

In [ ]:

#tech mahindra experience per quartile

llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print(i[i.TECHM_EXPERIENCE<=2].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())

In [ ]:

# (fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts()>3).sum(),fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts()
# fd1[fd1.TARGET_VALUE=='I'][fd1.TOTAL_EXPERIENCE>2].TOTAL_PROMOTIONS.value_counts(),fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.mean()

j=1
for i in llist:
    print('QUARTER NUMBER ',str(j))
    print('the percentage of total inexperienced and experienced',i[i.TOTAL_EXPERIENCE<=2].TOTAL_EXPERIENCE.value_counts().sum()*100/i.TOTAL_EXPERIENCE.value_counts().sum(),i[i.TOTAL_EXPERIENCE>2].TOTAL_EXPERIENCE.value_counts().sum()*100/i.TOTAL_EXPERIENCE.value_counts().sum())
    print('percent of the inexperienced who are leaving',i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts().sum()*100/i[i.TOTAL_EXPERIENCE<=3].TOTAL_EXPERIENCE.value_counts().sum())
    print('percent of the experienced who are leaving',i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts().sum()*100/i[i.TOTAL_EXPERIENCE>3].TOTAL_EXPERIENCE.value_counts().sum())
    print('-------------------')
    j+=1

In [ ]:

j=1
for i in llist:
  print('for QUARTER ',str(j))
  print((i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE<=3].TOTAL_PROMOTIONS.value_counts()*100)/i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE<=3].TOTAL_PROMOTIONS.value_counts().sum())
  print((i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE>3].TOTAL_PROMOTIONS.value_counts()*100)/i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE>3].TOTAL_PROMOTIONS.value_counts().sum())
  print('-------------------------')
  j+=1

In [ ]:
def stripplott(variable):
  f,ax=plt.subplots(figsize=(18,18))
  sb.set(rc={'figure.figsize':(16,20)})
  sb.stripplot(x='TARGET_VALUE', y=variable, data=fd1, alpha=0.3, jitter=True);
  # plt.plot()
  display()
  stripplott("TOTAL_EXPERIENCE")

In [ ]:
f,ax=plt.subplots(figsize=(18,18))
sb.set(rc={'figure.figsize':(100,50)})
# sb.countplot(x='BAND',data=fd3)
sb.factorplot(x='TARGET_VALUE',col='TOTAL_PROMOTIONS',kind='count',data=fd1)
# fd1.groupby('NO_OF_LEAVES').BAND.hist(alpha=0.6)
# fig, ax = plt.subplots(1, 2, figsize=(300, 100))
# sb.kdeplot(fd1.loc[fd1['TARGET_VALUE'] == 'I', 'NO_OF_MANAGER_REPORTEES'], ax=ax[0], label='INACTIVE(0)')
# sb.kdeplot(fd1.loc[fd1['TARGET_VALUE'] == 'A', 'NO_OF_MANAGER_REPORTEES'], ax=ax[0], label='ACTIVE(1)')
# plt.show()
display()

# PRE - PROCESSING

In [ ]:
#retain original copy
#data1 = validation_set.copy()
data1 = training_set.copy()

#validation_set = pd.concat([fd1,fd2,fd3,fd4,fd5,fd6]).drop_duplicates().reset_index(drop=True)
#data = fd8[fd8.TARGET_VALUE=='I'].copy()
#data1 = fd8.copy()
#llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]

In [ ]:
data1.rename(columns={'EMP_STATUS':'TARGET_VALUE'},inplace=True)

In [ ]:
data1['TARGET_VALUE'].value_counts()

In [ ]:
data1.tail()

In [ ]:

cat_df = data1.select_dtypes(include=['object']).copy()
cat_df.head()
#df2 = spark.createDataFrame(feature_to_use)

display(cat_df)

In [ ]:

data1['PROMOTION_FLAG']=data1.apply(lambda data1 : 1 if data1.LAST_PROMOTION_DATE.year>2016 and data1.LAST_PROMOTION_DATE.month<6 else 0,axis=1)
'''for i in llist:
    #fd1['PROMOTION_FLAG']=fd1.apply(lambda fd1 : 1 if fd1.LAST_PROMOTION_DATE.year>2016 and fd1.LAST_PROMOTION_DATE.month<6 else 0,axis=1)
    i['PROMOTION_FLAG']=i.apply(lambda i : 1 if i.LAST_PROMOTION_DATE.year>2016 and i.LAST_PROMOTION_DATE.month<6 else 0,axis=1)'''

In [ ]:
data1['NO_OF_LEAVES'].isnull().sum()

In [ ]:

lllist = [df1['NO_OF_LEAVES'],df2['NO_OF_LEAVES'],df3['NO_OF_LEAVES'],df4['NO_OF_LEAVES'],df5['NO_OF_LEAVES'],df6['NO_OF_LEAVES']]
lilist = [df1['Leaves_avg'],df2['Leaves_avg'],df3['Leaves_avg'],df4['Leaves_avg'],df5['Leaves_avg'],df6['Leaves_avg']]
for i in lllist:
  for j in lilist:
    if lllist.index(i)=lilist.index(j):
      #for i<index(i):
        j = i - sum(i)/index(j)
      
      


In [ ]:
data1.groupby('MASK_EMPLOYEEID').agg({'NO_OF_LEAVES':'sum','ID':'count'})

In [ ]:
data1['avg'] = data1.groupby('MASK_EMPLOYEE_ID').data1[['NO_OF_LEAVES']].mean(axis=1)
#data1.groupby('MASK_EMPLOYEEID').agg({'NO_OF_LEAVES':'sum','ID':'count'})

In [ ]:
def leave_fun(df):
  df1 = data1[(data1['Quarter'] <= df['Quarter']) & (data1['Year'] == df['Year']) & (data1['MASK_EMPLOYEEID'] == df['MASK_EMPLOYEEID'])]
  leave = df1['NO_OF_LEAVES'].sum() / df1['ID'].nunique()
  avg_leave = df['NO_OF_LEAVES'] - leave
  
  return avg_leave

In [ ]:
def progress_coroutine(print_on = 10):
    print ("Starting progress monitor")

    iterations = 0
    while True:
        yield
        iterations += 1
        if (iterations % print_on == 0):
            print ("{} iterations done".format(iterations))

def percentage_coroutine(to_process, print_on_percent = 0.10):
    print ("Starting progress percentage monitor")

    processed = 0
    count = 0
    print_count = to_process*print_on_percent
    while True:
        yield
        processed += 1
        count += 1
        if (count >= print_count):
            count = 0
            pct = (float(processed)/float(to_process))*100

            print ("{}% finished".format(pct))

def trace_progress(func, progress = None):
    def callf(*args, **kwargs):
        if (progress is not None):
            progress.send(None)

        return func(*args, **kwargs)

    return callf


In [ ]:
co1 = progress_coroutine()
co2 = percentage_coroutine(len(data1))
data1['Avg_leaves'] = data1.apply(trace_progress(leave_fun, progress = co1), axis = 1)

In [ ]:
'''df2['Leaves_avg'] = df2['NO_OF_LEAVES']-((df1['NO_OF_LEAVES']+df2['NO_OF_LEAVES'])/2)
df.sum(axis = 1, skipna = True)

mylist = [['name','party','city','0','0','1'],['name','party','city','1','1','1']]

print sum([sum(map(int, item[3:])) for item in mylist])  # prints 4


z = []
for i in list_df:
for index, row in i.NO_OF_LEAVES.iterrows():
...     z.append(row.x + row.y)
for i in list_df:
  for j in list_df:
  
    if (list_df.index(j)) <= (list_df.index(i)):
      x = (sum(j['NO_OF_LEAVES']))

       #i['Leaves_avg'] = i['NO_OF_LEAVES']-((sum(j['NO_OF_LEAVES']))/(list_df.index(i)+1))#sum([sum(map(int, item[3:])) for item in mylist]) 
'''

      
      
''' for j in lllist_df:
        if index(j)<=index(i):
          i = i - sum(j)/index(i)'''
'''df1['Leaves_avg'] = df1['NO_OF_LEAVES']
df2['Leaves_avg'] = df2['NO_OF_LEAVES'] - ((df1['NO_OF_LEAVES']+df2['NO_OF_LEAVES'])/2)
df3['Leaves_avg'] = df3['NO_OF_LEAVES'] - ((df1['NO_OF_LEAVES']+df2['NO_OF_LEAVES']+df3['NO_OF_LEAVES'])/3)'''

In [ ]:
data1['Avg_leaves']

In [ ]:
df3['Leaves_avg']

In [ ]:
df3['NO_OF_LEAVES']

In [ ]:
#data2=fd1.copy()
#data2.info()
data1['CITY_FLAG']=data1.apply(lambda data1 : 1 if data1.HIRE_LOCATION_CITY==data1.CURRENT_LOCATION_CITY else 0,axis=1 )

'''for i in llist:
    i['CITY_FLAG']=i.apply(lambda i : 1 if i.HIRE_LOCATION_CITY==i.CURRENT_LOCATION_CITY else 0,axis=1 )'''
#data1.CITY_FLAG.value_counts()

In [ ]:
'''data1.PRIMARY_SKILL.fillna(data1.SECONDARY_SKILL.str[0],inplace=True)
data["PRIMARY_SKILL"]= data["PRIMARY_SKILL"].str.replace(np.nan, "New Boston", case = False)
data["PRIMARY_SKILL"]'''

In [ ]:
data1.CURRENT_RATING.fillna(data1.LAST_RATING,inplace=True)
'''for i in llist:
    i.CURRENT_RATING.fillna(i.LAST_RATING,inplace=True)'''

In [ ]:
data1.CURRENT_RATING.fillna(data1.ALL_PREVIOUS_RAINGS.str[-1],inplace=True)
'''for i in llist:
    i.CURRENT_RATING.fillna(i.ALL_PREVIOUS_RAINGS.str[-1],inplace=True)'''

In [ ]:
data1.columns

In [ ]:
import pickle

pickle_in = open("band.pickle","rb")
dict_band = pickle.load(pickle_in)
#print(dict_band)

pickle_in = open("education.pickle","rb")
dict_education = pickle.load(pickle_in)
#print(dict_education)

pickle_in = open("rating.pickle","rb")
dict_rating = pickle.load(pickle_in)
#print(dict_rating)

pickle_in = open("education_name.pickle","rb")
dict_edu_name = pickle.load(pickle_in)
#print(dict_edu_name)

pickle_in = open("gender.pickle","rb")
dict_gender = pickle.load(pickle_in)
#print(dict_gender)

pickle_in = open("generic.pickle","rb")
dict_generic = pickle.load(pickle_in)
#print(dict_generic)

pickle_in = open("target.pickle","rb")
dict_target = pickle.load(pickle_in)
#print(dict_target)

#print(example_dict[3])

In [ ]:
data1.head()

In [ ]:
data1.columns

In [ ]:
'''quarter_dict = {3: 1, 6: 2, 10: 3, 12: 4} # quarter dict
data1['Year'] = data1[data1['TARGET_VALUE'] == 'I']['HIRE_EXIT_DATE'].max().year
data1['Quarter'] = pd.Series(data1[data1['TARGET_VALUE'] == 'I']['HIRE_EXIT_DATE'].max().month).apply(quarter_dict.get)[0]
data1['Month'] = data1[data1['TARGET_VALUE'] == 'I']['HIRE_EXIT_DATE'].max().month

data1['ID']=data1['Quarter'].astype(str)+'_'+data1['Year'].astype(str)'''

In [ ]:
data1['HIGHEST_EDUCATION'].replace(dict_edu_name, inplace=True)
'''for i in llist:
    i['HIGHEST_EDUCATION'].replace(dict_edu_name, inplace=True)'''

In [ ]:
data1['BAND'].replace(dict_band, inplace=True)
data1['CURRENT_RATING'].replace(dict_rating, inplace=True)
data1['LAST_RATING'].replace(dict_rating, inplace=True)
data1['MANAGER_RATING'].replace(dict_rating, inplace=True)
data1['HIGHEST_EDUCATION'].replace(dict_education, inplace=True)
data1['GENDER'].replace(dict_gender, inplace=True)
data1['GRIEVANCE_FLG'].replace(dict_generic, inplace=True)
data1['BILLABILITY_STATUS'].replace(dict_generic, inplace=True)
data1['RESIGNATION_FLG'].replace(dict_generic, inplace=True)
data1['TARGET_VALUE'].replace(dict_target, inplace=True)

'''
for i in llist:
    i['BAND'].replace(dict_band, inplace=True)
    i['CURRENT_RATING'].replace(dict_rating, inplace=True)
    i['LAST_RATING'].replace(dict_rating, inplace=True)
    i['MANAGER_RATING'].replace(dict_rating, inplace=True)
    i['HIGHEST_EDUCATION'].replace(dict_education, inplace=True)'''

In [ ]:

print(data1.dtypes)

In [ ]:
#Conversion of object to category 
for i in data1:
    if data1[i].dtype==object:
        data1[i] = data1[i].astype('category').cat.codes


In [ ]:
llisst = ['EMPLOYEE_AGE','NO_OF_LEAVES','TOTAL_REPORTEES','NO_OF_MANAGER_REPORTEES','TECHM_EXPERIENCE','TOTAL_EXPERIENCE','NO_OF_PREVIOUS_EMPLOYERS','LAST_ONSITE_TRAVEL_DAYS','TOTAL_PROMOTIONS','MANAGER_RATING','CURRENT_RATING','LAST_RATING','REWARDS','QUARTILE','TENURE_LAST_EMPLOPYER','LATEST_HIKE_PERCENT','LAST_HIKE_PERCENT','HIGHEST_EDUCATION','CITY_FLAG','Month','GRIEVANCE_FLG','BILLABILITY_STATUS']
for j in llisst:
      data1[j] = data1[j].fillna(0.0).astype(int)


In [ ]:
data1.dtypes

In [ ]:


corr = data1.corr()
f,ax=plt.subplots(figsize=(10,10))
#b.heatmap(corr,annot=True,linewidths=0.5,linecolor="green",fmt=".1f",ax=ax)
sb.heatmap(corr,annot=False,linewidths=0.5,linecolor="green",fmt=".1f",ax=ax)
#plt.show()
display()


In [ ]:
'''#data1 = pd.get_dummies(data1, columns=['carrier'], prefix = ['carrier'])
data1 = pd.get_dummies(data1, columns=['GENDER','DEPARTMENT_ID','PRIMARY_SKILL','SECONDARY_SKILL','CURRENT_LOCATION_CITY','BASE_LOCATION_CITY','HIRE_LOCATION_CITY','RESIGNATION_FLG','BILLABILITY_STATUS','ALL_PREVIOUS_RAINGS','OFFICERCODE_DESC','GRIEVANCE_FLG'], prefix = ['GENDER','DEPARTMENT_ID','PRIMARY_SKILL','SECONDARY_SKILL','CURRENT_LOCATION_CITY','BASE_LOCATION_CITY','HIRE_LOCATION_CITY','RESIGNATION_FLG','BILLABILITY_STATUS','ALL_PREVIOUS_RAINGS','OFFICERCODE_DESC','GRIEVANCE_FLG'])

print(data1.head())
'''

In [ ]:

#to put target variable in last
dft = data1.pop('TARGET_VALUE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['TARGET_VALUE']=dft # add b series as a 'new' column.
#df['x']=df2 # add b series as a 'new' column.


In [ ]:

dft = data1.pop('HIRE_EXIT_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['HIRE_EXIT_DATE']=dft


dft = data1.pop('LAST_PROMOTION_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_PROMOTION_DATE']=dft

dft = data1.pop('MANAGER_LAST_PROMOTION') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['MANAGER_LAST_PROMOTION']=dft


dft = data1.pop('LAST_ONSITE_TRAVEL_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_ONSITE_TRAVEL_DATE']=dft



In [ ]:
data1.columns

In [ ]:
#Removing irrelevant features

threshold = 90
# Absolute value correlation matrix
corr_matrix = (data1.corr().abs())*100
corr_matrix.head(30)

# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))
print(to_drop)


In [ ]:
#remove = ['LAST_ONSITE_TRAVEL_DATE','MANAGER_LAST_PROMOTION','LAST_PROMOTION_DATE','HIRE_EXIT_DATE','TARGET_VALUE']
remove = ['LAST_ONSITE_TRAVEL_DATE','MANAGER_LAST_PROMOTION','LAST_PROMOTION_DATE','HIRE_EXIT_DATE','RESIGNATION_FLG','CURRENT_LOCATION_CITY','Month','HIRE_LOCATION_CITY']

data1.drop(remove, axis=1, inplace=True)


In [ ]:

import pandas as pd
data1.to_pickle('preprocessed.pkl')    #to save the dataframe, df to 123.pkl

In [ ]:
'''import pandas as pd
data1.to_pickle('test_valid.pkl')  '''

#PRE processing completed

In [ ]:
#model.save('Minst_Lstm.model')

In [ ]:

# Basic statistics of categorical features
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  print(i.describe(include=[np.object]))
#cat_col_names = data.select_dtypes(include=[np.object]).columns.tolist() # Get categorical feature names
#cat_col_names

In [ ]:
#Attrition Target Variable Distribution
lllist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in lllist:
  attrition_freq = i[['TARGET_VALUE']].apply(lambda x: x.value_counts())
  attrition_freq['frequency_percent'] = round((100 * attrition_freq / attrition_freq.sum()),2)

  print(attrition_freq)


In [ ]:
def distribution(variable):
  for i in llist:
      print(i[variable].value_counts(),(i[i.TARGET_VALUE=='I'][variable].value_counts()*100)/i[variable].value_counts(),i[i.TARGET_VALUE=='I'][variable].value_counts())

In [ ]:
distribution('DEPARTMENT_ID')

In [ ]:
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  print(distribution('OFFICERCODE_DESC'))

In [ ]:
 """To understand how each features are impacting the attrition indicator, we need to first handle the null / missing values, otherwise our observations might not be accurate and will lead to wrong conclusions.
"""
for i in llist:
    null_feat_df = pd.DataFrame()
    null_feat_df['Null Count'] = i.isnull().sum().sort_values(ascending=False)
    null_feat_df['Null Pct'] = null_feat_df['Null Count'] / float(len(i))

    null_feat_df = null_feat_df[null_feat_df['Null Pct'] > 0]


    total_null_feats = null_feat_df.shape[0]
    null_feat_names = null_feat_df.index
    print('Total number of features having null values: ', total_null_feats)
    del null_feat_df

In [ ]:
#filling with 0
for i in llist:
    i["LATEST_HIKE_PERCENT"].fillna("0", inplace = True) 
    i["LAST_HIKE_PERCENT"].fillna("0", inplace = True)

    i['EMPLOYEE_AGE'].fillna(i['EMPLOYEE_AGE'].mode()[0], inplace=True) 
#EMPLOYEE_AGE 

In [ ]:

plt.figure(1) 
plt.subplot(221) 
data['GENDER'].value_counts(normalize=True).plot.bar(figsize=(20,10), title= 'Gender') 
plt.subplot(222) 
data['BAND'].value_counts(normalize=True).plot.bar(title= 'Band') 
plt.subplot(223) 
data['REWARDS'].value_counts(normalize=True).plot.bar(title= 'REWARDS') 
plt.subplot(224) 
#plt.show()
display()

In [ ]:
# Segregating data set based on Attrition value

#Attrition across Job Roles
fig, ax6 = plt.subplots(1,2, figsize=(24,10))
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
sb.countplot(x='BAND', hue='GENDER', data = attr_yes, palette="Set3", ax = ax6[1])
sb.countplot(x='BILLABILITY_STATUS', hue='GENDER', data = attr_yes,palette="Set3", ax = ax6[0])
plt.show()
display()
    

In [ ]:
#
def plot_count(feature, title,size=1):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(data))
    g = sb.countplot(data[feature], order = data[feature].value_counts(normalize=True).index[:20], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()   
    display()

In [ ]:
plot_count('CURRENT_RATING','current rating')

In [ ]:
#
plt.figure(figsize=(15, 8))
cols = ['LAST_ONSITE_TRAVEL_DAYS', 'REWARDS', 'TOTAL_PROMOTIONS', 'TENURE_LAST_EMPLOPYER', 'NO_OF_PREVIOUS_EMPLOYERS']
uniques = [len(data[col].unique()) for col in cols]
sb.set(font_scale=1.2)
sb.color_palette("Blues")
ax = sb.barplot(cols, uniques,log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 
# for col, uniq in zip(cols, uniques):
#     ax.text(col, uniq, uniq, color='black', ha="center")
display()

In [ ]:
def divisionplot(variable):
  var=pd.crosstab(data[variable],data['QUARTILE'])
  var.div(var.sum(1).astype(float), axis=0).plot(kind="bar", stacked=False, figsize = (10,4)) 
  #plt.show()
  display()

In [ ]:

divisionplot('BAND')

In [ ]:

sb.jointplot(x="TOTAL_PROMOTIONS", y="TOTAL_EXPERIENCE", data=data,  ratio=3, color="r")
plt.show()
display()



In [ ]:
#main_data = fd4.copy()
listdf = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in listdf:
  print(i['CURRENT_RATING'].value_counts() , i['CURRENT_RATING'][i['TARGET_VALUE'] == 'I'].value_counts())
  #print(i['CURRENT_RATING'][i['TARGET_VALUE'] == 'I'].value_counts())
  

In [ ]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['CURRENT_RATING']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['CURRENT_RATING']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)


In [ ]:
data['AGE_Interval'] = pd.cut(data['EMPLOYEE_AGE'], 6, labels=['19-25','25-35', '35-45', '45-55', '55-65','65+'])

In [ ]:

age=pd.DataFrame(data.groupby("AGE_Interval")[["TOTAL_REPORTEES","NO_OF_LEAVES","NO_OF_MANAGER_REPORTEES","TECHM_EXPERIENCE","TOTAL_EXPERIENCE","LAST_HIKE_PERCENT","NO_OF_PREVIOUS_EMPLOYERS","TENURE_LAST_EMPLOPYER","TOTAL_PROMOTIONS","LAST_ONSITE_TRAVEL_DAYS","REWARDS","LATEST_HIKE_PERCENT","HIGHEST_EDUCATION"]].mean())
age["Count"]=data.AGE_Interval.value_counts(dropna=False)
age.reset_index(level=0, inplace=True)
age.head()

In [ ]:
plt.figure(figsize=(6,6))
ax=sb.barplot(x=age.AGE_Interval,y=age.Count)
plt.xticks(rotation=90)
plt.xlabel("Age")
plt.ylabel("Counts")
plt.title("Age Counts")
plt.show()
display()

In [ ]:

data['AGE_Interval'].value_counts()

In [ ]:
plt.figure(figsize=(6,6))
ax=sb.barplot(x=age.AGE_Interval,y=age.TOTAL_PROMOTIONS,palette = sb.cubehelix_palette(len(age.index)))
plt.xticks(rotation=90)
plt.xlabel("Age")
plt.ylabel("TOTAL_PROMOTIONS")
plt.title("TOTAL_PROMOTIONS According to Age")
plt.show()
display()

In [ ]:
"""
"""
labels=data.AGE_Interval.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=(data[data.TARGET_VALUE=='I'].AGE_Interval.value_counts()*100)/data.AGE_Interval.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("AGE interval Counts",color="saddlebrown",fontsize=15)
display()


In [ ]:
age1=pd.DataFrame(data.groupby("EMPLOYEE_AGE")[["TOTAL_REPORTEES","NO_OF_LEAVES","NO_OF_MANAGER_REPORTEES","TECHM_EXPERIENCE","TOTAL_EXPERIENCE","LAST_HIKE_PERCENT","NO_OF_PREVIOUS_EMPLOYERS","TENURE_LAST_EMPLOPYER","TOTAL_PROMOTIONS","LAST_ONSITE_TRAVEL_DAYS","REWARDS","LATEST_HIKE_PERCENT","HIGHEST_EDUCATION"]].mean())
age1["Count"]=data.EMPLOYEE_AGE.value_counts(dropna=False)
age1.reset_index(level=0, inplace=True)
age1.head()

In [ ]:
agenorm=age1.apply(lambda x: x/max(x))
agenorm.head()

In [ ]:
ageb=data.EMPLOYEE_AGE.value_counts().index
f,ax=plt.subplots(figsize=(15,15))
sb.pointplot(y=agenorm.LAST_HIKE_PERCENT,x=ageb,color="purple",alpha=0.8)
sb.pointplot(y=agenorm.TOTAL_PROMOTIONS,x=ageb,color="sandybrown",alpha=0.8)
plt.text(5,0.65,"LAST_HIKE_PERCENT",color="purple",fontsize=15,style="italic")
plt.text(5,0.63,"TOTAL_PROMOTIONS",color="sandybrown",fontsize=15,style="italic")
plt.xlabel("Age",fontsize=15,color="darkred")
plt.ylabel("Values",fontsize=15,color="darkred")
plt.title("LAST_HIKE_PERCENTe VS JTOTAL_PROMOTIONS",fontsize=15,color="darkred")
plt.grid()
display()

In [ ]:
ageb=data.EMPLOYEE_AGE.value_counts().index
f,ax=plt.subplots(figsize=(15,15))
sb.pointplot(y=agenorm.LATEST_HIKE_PERCENT,x=ageb,color="purple",alpha=0.8)
sb.pointplot(y=agenorm.TOTAL_PROMOTIONS,x=ageb,color="sandybrown",alpha=0.8)
plt.text(5,0.65,"LATEST_HIKE_PERCENT",color="purple",fontsize=15,style="italic")
plt.text(5,0.63,"TOTAL_PROMOTIONS",color="sandybrown",fontsize=15,style="italic")
plt.xlabel("Age",fontsize=15,color="darkred")
plt.ylabel("Values",fontsize=15,color="darkred")
plt.title("LAST_HIKE_PERCENTe VS JTOTAL_PROMOTIONS",fontsize=15,color="darkred")
plt.grid()
display()

In [ ]:
f,ax = plt.subplots(figsize = (15,10))
sb.kdeplot(agenorm.REWARDS,agenorm.LAST_ONSITE_TRAVEL_DAYS,shade=True,cut=1)
display()

In [ ]:
f,ax = plt.subplots(figsize = (15,10))
sb.kdeplot(agenorm.TOTAL_PROMOTIONS,agenorm.LAST_ONSITE_TRAVEL_DAYS,shade=True,cut=1)
display()

In [ ]:
promotions=pd.DataFrame(data.groupby("BAND").TOTAL_PROMOTIONS.mean().sort_values(ascending=False))
plt.figure(figsize=(5,5))
ax=sb.barplot(x=promotions.index,y=promotions.TOTAL_PROMOTIONS)
plt.xticks(rotation=90)
plt.xlabel("BAND")
plt.ylabel("TOTAL_PROMOTIONS")
plt.title("BAND with TOTAL_PROMOTIONS")
plt.show()
display()

In [ ]:
band=pd.DataFrame(data.groupby("BAND")["EMPLOYEE_AGE","NO_OF_LEAVES","NO_OF_PREVIOUS_EMPLOYERS","NO_OF_MANAGER_REPORTEES","LAST_ONSITE_TRAVEL_DAYS","LATEST_HIKE_PERCENT","LAST_HIKE_PERCENT","NO_OF_MANAGER_REPORTEES","TENURE_LAST_EMPLOPYER","TECHM_EXPERIENCE","TOTAL_EXPERIENCE"].mean())
band

In [ ]:
f,ax = plt.subplots(figsize = (9,10))
sb.barplot(x=band.LATEST_HIKE_PERCENT,y=band.index,color='green',alpha = 0.5,label='LATEST_HIKE_PERCENT' )
sb.barplot(x=band.LAST_ONSITE_TRAVEL_DAYS,y=band.index,color='blue',alpha = 0.7,label='LAST_ONSITE_TRAVEL_DAYS')
sb.barplot(x=band.TECHM_EXPERIENCE,y=band.index,color='cyan',alpha = 0.6,label='Years At Company')
sb.barplot(x=band.NO_OF_PREVIOUS_EMPLOYERS,y=band.index,color='yellow',alpha = 0.6,label='NO_OF_PREVIOUS_EMPLOYERS')
sb.barplot(x=band.NO_OF_LEAVES,y=band.index,color='red',alpha = 0.6,label='NO_OF_LEAVES')

ax.legend(loc='right',frameon = True)     
ax.set(xlabel='Values', ylabel='band',title = "band with Different Features")
plt.show()
display()

In [ ]:
g = sb.pairplot(data, vars=["TOTAL_PROMOTIONS", "REWARDS"],hue="BAND",size=5)
display()

In [ ]:

f,ax = plt.subplots(figsize = (5,5))
sb.boxplot(x="GENDER",y="EMPLOYEE_AGE",hue="QUARTILE",data=data,palette="Paired")
display()

In [ ]:
f,ax = plt.subplots(figsize = (15,10))
ax.legend(frameon=False, loc='lower center', ncol=2)

sb.boxplot(x="GENDER",y="EMPLOYEE_AGE",hue="BAND" ,data=data,palette="hls")
display()

In [ ]:

f,ax = plt.subplots(figsize = (5,5))
sb.pointplot(x="GENDER", y="TECHM_EXPERIENCE", hue="TARGET_VALUE", data=data,
              palette={"I": "blue", "A": "pink"},
              markers=["*", "o"], linestyles=["-", "--"]);
display()
## Inference
# Males and Females within the ranges for 'TotalWorkingYears' of 11 to 13 are less likely to quit

In [ ]:
data['TECHM_Interval'] = pd.cut(data['TECHM_EXPERIENCE'], 6, labels=['0-5','5-10', '10-15', '15-20', '20-25','25+'])

In [ ]:
f,ax = plt.subplots(figsize = (5,5))
#plt.subplots(figsize=(15,5))
sb.countplot(data.TECHM_Interval)
display()

In [ ]:
data['HIGHEST_EDUCATION'].value_counts()


#print(data['HIGHEST_EDUCATION'].value_counts())

In [ ]:
data1

In [ ]:
data1['EDUCATION_CATEGORIES'].value_counts()

In [ ]:
f,ax = plt.subplots(figsize = (11,5))
#plt.subplots(figsize=(15,5))
sb.countplot(data.EDUCATION_CATEGORIES)
display()

In [ ]:
labels=data.EDUCATION_CATEGORIES.value_counts().index
colors=["olive","orange","hotpink","slateblue","y","lime","b"]
explode=[1,0,0,0,1,1]
sizes=data.EDUCATION_CATEGORIES.value_counts() 
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,colors=colors,autopct="%1.1f%%")
plt.title("Education Field Counts",color="saddlebrown",fontsize=15)
display()


In [ ]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  i['LeavesInterval'] = pd.cut(i['NO_OF_LEAVES'][i['NO_OF_LEAVES']<50], 5, labels=['<10', '<20', '<30', '<40','<50+'])
  freq = i[['LeavesInterval']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['LeavesInterval']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)

In [ ]:
#data['LeavesInterval'].value_counts()
data['LeavesInterval'] = pd.cut(i['NO_OF_LEAVES'][i['NO_OF_LEAVES']<50], 5, labels=['<10', '<20', '<30', '<40','<50+'])

In [ ]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['DEPARTMENT_ID']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['DEPARTMENT_ID']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  #print(freq)
  print(freq["frequency_percent"].max())

In [ ]:
# library and dataset
#import seaborn as sns
#df = sns.load_dataset('iris')
f,ax = plt.subplots(figsize = (15,10))
# plot of 2 variables
sb.kdeplot(data['TOTAL_REPORTEES'], shade=True, color="r")
sb.kdeplot(data['NO_OF_MANAGER_REPORTEES'], shade=True, color="b")
#sns.plt.show()
display()

In [ ]:
# library and dataset
#import seaborn as sns
#df = sns.load_dataset('iris')
f,ax = plt.subplots(figsize = (5,5))
# plot of 2 variables
sb.kdeplot(data['TOTAL_PROMOTIONS'], shade=True, color="r")
sb.kdeplot(data['REWARDS'], shade=True, color="b")
#sns.plt.show()
display()

In [ ]:
data['HIRE_LOCATION_CITY'].value_counts()

In [ ]:
idx1 = pd.Index(data.CURRENT_LOCATION_CITY)
idx2 = pd.Index(data.BASE_LOCATION_CITY)
idx3 = pd.Index(data.HIRE_LOCATION_CITY)
#main_data['unique'] = idx1.difference(idx2).values
print(idx1.difference(idx2).values)
print(idx2.difference(idx1).values)
print(idx3.difference(idx1).values)
#print(idx3.difference(idx2).values)
#array([3, 6])

In [ ]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['HIRE_LOCATION_CITY']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['HIRE_LOCATION_CITY']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)

In [ ]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['HIRE_EXIT_DATE']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['HIRE_EXIT_DATE']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)
  
  
  

In [ ]:
data.columns

In [ ]:
features=['LeavesInterval','TECHM_Interval','CURRENT_RATING','AGE_Interval', 'TOTAL_PROMOTIONS','DEPARTMENT_ID ','QUARTILE']
fig=plt.subplots(figsize=(10,15))
for i, j in enumerate(features):
    plt.subplot(4, 2, i+1)
    plt.subplots_adjust(hspace = 1.0)
    sb.countplot(x=j,data = data)
    plt.xticks(rotation=90)
    plt.title("No. of employee")
    display()

In [ ]:
# pie chart of workers
labels = ['M', 'F']
sizes = [data['GENDER'].value_counts()[0],
         data['GENDER'].value_counts()[1]
        ]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True)
ax1.axis('equal')
plt.title('Gender Pie Chart', fontsize=20)
plt.show()
display()

## Inference
# Proportion of male is higher than female associates

In [ ]:
fig=plt.subplots(figsize=(5,5))
sb.factorplot(data=data,y='TOTAL_PROMOTIONS',x='BAND',size=7,aspect=2,kind='point')
display()


In [ ]:
x

In [ ]:
"""
# Plotting the KDEplots
f, axes = plt.subplots(3, 3, figsize=(10, 8), 
                       sharex=False, sharey=False)

# Defining our colormap scheme
s = np.linspace(0, 3, 7)
cmap = sb.cubehelix_palette(start=0.0, light=1, as_cmap=True)

# Generate and plot
x = data['EMPLOYEE_AGE'].values
y = data['TECHM_EXPERIENCE'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, cut=5, ax=axes[0,0])
axes[0,0].set( title = 'Age Vs TechMExperience')
#display()
cmap = sb.cubehelix_palette(start=0.333333333333, light=1, as_cmap=True)

# Generate and plot
x = data['EMPLOYEE_AGE'].values
y = data['NO_OF_LEAVES'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,1])
axes[0,1].set( title = 'Age Vs no_of_leaves')
#display()

cmap = sb.cubehelix_palette(start=0.666666666667, light=1, as_cmap=True)
# Generate and plot
x = data['NO_OF_PREVIOUS_EMPLOYERS'].values
y = data['EMPLOYEE_AGE'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,2])
axes[0,2].set( title = 'No_of_prev_employers Vs Age')
display()
'''
cmap = sns.cubehelix_palette(start=1.0, light=1, as_cmap=True)
# Generate and plot
x = attrition['DailyRate'].values
y = attrition['DistanceFromHome'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,0])
axes[1,0].set( title = 'Daily Rate against DistancefromHome')

cmap = sns.cubehelix_palette(start=1.333333333333, light=1, as_cmap=True)
# Generate and plot
x = attrition['DailyRate'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,1])
axes[1,1].set( title = 'Daily Rate against Job satisfaction')

cmap = sns.cubehelix_palette(start=1.666666666667, light=1, as_cmap=True)
# Generate and plot
x = attrition['YearsAtCompany'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,2])
axes[1,2].set( title = 'Daily Rate against distance')

cmap = sns.cubehelix_palette(start=2.0, light=1, as_cmap=True)
# Generate and plot
x = attrition['YearsAtCompany'].values
y = attrition['DailyRate'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,0])
axes[2,0].set( title = 'Years at company against Daily Rate')

cmap = sns.cubehelix_palette(start=2.333333333333, light=1, as_cmap=True)
# Generate and plot
x = attrition['RelationshipSatisfaction'].values
y = attrition['YearsWithCurrManager'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,1])
axes[2,1].set( title = 'Relationship Satisfaction vs years with manager')

cmap = sns.cubehelix_palette(start=2.666666666667, light=1, as_cmap=True)
# Generate and plot
x = attrition['WorkLifeBalance'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,2])
axes[2,2].set( title = 'WorklifeBalance against Satisfaction')
f.tight_layout()
dispaly()
'''
"""

In [ ]:
"""
train_small = data# train.sample(frac=0.2).copy() # not small for now
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sb.kdeplot(train_small.loc[data['BILLABILITY_STATUS'] == 'N', 'EMPLOYEE_AGE'], ax=ax[0], label='NoPay(0)')
sb.kdeplot(train_small.loc[data['BILLABILITY_STATUS'] == 'Y', 'EMPLOYEE_AGE'], ax=ax[0], label='HasPay(1)')

#\train_small.loc[data['HasDetections'] == 0, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
#train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
#ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()
display()
"""
'''
'''

In [ ]:
"""
'''
# Segregating data set based on Attrition value
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
'''
attrition = data[(data['TARGET_VALUE'] == 'A' )]
no_attrition = data[(data['TARGET_VALUE'] == 'I')]
'''
# Segregating data set based on Attrition value
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
#Add columns to store discrete variables for Salary range and alcohol levels
#Adding discrete valriables
data['LeavesInterval'] = pd.cut(data['NO_OF_LEAVES'], 5, labels=['<9', '<15', '<24', '<32', '<33+'])
data['TechMExpLvl'] = pd.cut(data['TECHM_EXPERIENCE'], 5, labels=['lvl1', 'lvl2', 'lvl3', 'lvl4', 'lvl5'])
data['traveldate'] = pd.cut(data['LAST_ONSITE_TRAVEL_DAYS'], 5, labels=['0to3', '3to6', '6to9', '9to12', '12+'])



#Factor Plot
sb.factorplot(x =   'TARGET_VALUE',     # Categorical
               y =   'NO_OF_LEAVES',          # Continuous
               hue = 'DEPARTMENT_ID',   # Categorical
               col = 'LeavesInterval',   # Categorical for graph columns
               col_wrap=3,           # Wrap facet after two axes
               kind = 'box',
               data = data)
plt.show()
display()


# Attrition count across Daily Rate, Years in company, Years since last promotion and level of stock options
fig, ax2 = plt.subplots(2,2, figsize=(10,10))
sb.countplot(x='LeavesInterval', data=attr_yes, ax = ax2[0,0])
sb.countplot(x='TechMExpLvl', data=attr_yes, ax = ax2[0,1])
sb.countplot(x='traveldate', data=attr_yes, ax = ax2[1,0])
#sb.countplot(x='StockOptionLevel', data=attr_yes, ax = ax2[1,1])
plt.show()
display()
'''
"""
"""
#Count Plot
# Attrition across Education level and corresponding percentage across the total number of employees
total = data.shape[0] 
hrfig = sb.countplot(x='HIGHEST_EDUCATION', hue = 'TARGET_VALUE', data = data)
#Above graph showcases the percentages of the employees across Education levels and corresponding attri
for p in hrfig.patches:
    height = p.get_height()
    hrfig.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height*100/total),
            ha="center") 
plt.show()
display()
"""


In [ ]:
'''from datetime import date
from datetime import datetime

dt = datetime.combine(date.today(), datetime.min.time())
print(dt)'''

In [ ]:
"""
# Attrition distribution bar plot

plot = attrition_freq[['frequency_percent']].plot(kind="bar");
plot.set_title("Attrition Distribution", fontsize=40);
plot.grid(color='lightgray', alpha=0.5);
display()
"""
"""Just by a quick inspection of the counts of the number of 'Yes' and 'No' in the target variable tells us that there is quite a large skew in target variable.
Therefore we have to keep in mind that there is quite a big imbalance in our target variable. Many statistical techniques have been put forth to treat imbalances in data (oversampling or undersampling).
In this notebook, I will use an oversampling technique known as SMOTE to treat this imbalance.
We will see the prediction model with and without SMOTE treatment for imbalance class issue."""


In [ ]:
"""
#Step 4 - Feature Engineering
Feature engineering is one aspect which provided a huge impact on the outcome rather than the model. Here, we try at creating new features with the existing variables we have based on my assumptions.

Step 4.1 - Addition of New Features
Tenure per job: Usually, people who have worked with many companies but for small periods at every organization tend to leave early as they always need a change of Organization to keep them going.
Years without Change: For any person, a change either in role or job level or responsibility is needed to keep the work exciting to continue. We create a variable to see how many years it has been for an employee without any sort of change using Promotion, Role and Job Change as a metric to cover different variants of change.
Compensation Ratio: Compa Ratio is the ratio of the actual pay of an Employee to the midpoint of a salary range. The salary range can be that of his/her department or organization or role. The benchmark numbers can be a organization’s pay or Industry average.


"""
emp_proc_df = data.copy() # Copy cleaned dataset for feature engineering

emp_proc_df['TenurePerJob'] = 0

for i in range(0, len(emp_proc_df)):
    if emp_proc_df.loc[i,'NO_OF_PREVIOUS_EMPLOYERS'] > 0:
        emp_proc_df.loc[i,'TenurePerJob'] = emp_proc_df.loc[i,'TOTAL_EXPERIENCE'] / emp_proc_df.loc[i,'NO_OF_PREVIOUS_EMPLOYERS']
"""
emp_proc_df['YearWithoutChange1'] = emp_proc_df['YearsInCurrentRole'] - emp_proc_df['YearsSinceLastPromotion']
emp_proc_df['YearWithoutChange2'] = emp_proc_df['TotalWorkingYears'] - emp_proc_df['YearsSinceLastPromotion']

monthly_income_median = np.median(emp_proc_df['MonthlyIncome'])
emp_proc_df['CompRatioOverall'] = emp_proc_df['MonthlyIncome'] / monthly_income_median

print('Dataset dimension: {} rows, {} columns'.format(emp_proc_df.shape[0], emp_proc_df.shape[1]))
"""









full_col_names = emp_proc_df.columns.tolist()
num_col_names = emp_proc_df.select_dtypes(include=[np.int64,np.float64]).columns.tolist() # Get numerical feature names

# Preparing list of ordered categorical features
for i in emp_proc_df:
    if emp_proc_df[i].dtype==object:
        num_cat_col_names = emp_proc_df[i]
#num_cat_col_names = ['DEPARTMENT_ID', 'TARGET_VALUE', 'OFFICERCODE_DESC', 'ALL_PREVIOUS_RAINGS', 'JobSatisfaction',
                     #'PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance', 'StockOptionLevel']

target = ['TARGET_VALUE']

num_col_names = list(set(num_col_names) - set(num_cat_col_names)) # Numerical features w/o Ordered Categorical features
cat_col_names = list(set(full_col_names) - set(num_col_names) - set(target)) # Categorical & Ordered Categorical features

print('Total number of numerical features: ', len(num_col_names))
print('Total number of categorical & ordered categorical features: ', len(cat_col_names))
cat_emp_df = emp_proc_df[cat_col_names]
num_emp_df = emp_proc_df[num_col_names]









#Transform Numerical Features
#In order to fix the skewness, let’s take the log for all numerical features with an absolute skew greater than 80% (Note: log+1, to avoid division by zero issues).

# Let's create dummy variables for each categorical attribute for training our calssification model
for col in num_col_names:
    if num_emp_df[col].skew() > 0.80:
        num_emp_df[col] = np.log1p(num_emp_df[col])

num_emp_df.head()





""" #Transform Categorical Features
Machine Learning model works only on numerical datasets, hence, we need to transform categorical features into numerical features.
One of the best strategy is to convert each category value into a new column and assigns 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set.
This approach is also called as "One Hot Encoding". We can use Pandas feature get_dummies to achieve this transformation.
There is another way to handled ordered categorical feature is to give ordered value based on their definitions i.e Low-Meidum-High would be 1-2-3. We can try this approach some other time. But, this can be evaluated to check the performance of the model.
"""
# Let's create dummy variables for each categorical attribute for training our calssification model
for col in cat_col_names:
    col_dummies = pd.get_dummies(cat_emp_df[col], prefix=col)
    cat_emp_df = pd.concat([cat_emp_df, col_dummies], axis=1)

# Use the pandas apply method to numerically encode our attrition target variable
attrition_target = emp_proc_df['TARGET_VALUE'].map({'I':1, 'A':0})

# Drop categorical feature for which dummy variables have been created

cat_emp_df.drop(cat_col_names, axis=1, inplace=True)
cat_emp_df.head()




# Correlation of Numerical Features against Attrition
num_corr_df = num_emp_df[['EMPLOYEE_AGE', 'NO_OF_MANAGER_REPORTEES', 'TOTAL_REPORTEES']]
corr_df = pd.concat([num_corr_df, attrition_target], axis=1)
corr = corr_df.corr()
display()
plt.figure(figsize = (10, 8))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sb.axes_style("white")
#sns.heatmap(data=corr, annot=True, mask=mask, square=True, linewidths=.5, vmin=-1, vmax=1, cmap="YlGnBu")
sb.heatmap(data=corr, annot=True, square=True, linewidths=.5, vmin=-1, vmax=1, cmap="YlGnBu")
plt.show()
display()






# Another way to check for correlation between attributes is to use Pandas’ scatter_matrix function,

from pandas.plotting import scatter_matrix

scatter_matrix(corr_df, figsize=(16, 10));
display()



# Concat the two dataframes together columnwise
final_emp_df = pd.concat([num_emp_df, cat_emp_df], axis=1)

print('Dataset dimension after treating categorical features with dummy variables: {} rows, {} columns'.format(final_emp_df.shape[0], final_emp_df.shape[1]))
final_emp_df.head()










"""
#Model Building and Validation
Since, we have to predict a binary class, we will be using classification models for training & predicting Employee Attrition. We need to keep in mind that our focus should be to have a better accuracy of predicting attrition i.e. Attrition = 1 which in confusion matrix will be "True Positive". However, we should not forget the prediction accuracy of not qualifying for attrition i.e. Attrition = 0 which will be "True Negative" in confusion matrix.

So, we need to focus on four parameters:

Accuracy: Overall, how often is the classifier correct? i.e {(TP+TN)/Total}
True Positive Rate: When it's actually yes, how often does it predict yes? default_ind = 1, {TP/Actual YES}, this is also known as "Sensitivity" or "Recall"
Precision: When it predicts yes, how often is it correct? i.e. {TP/(TP+FP)}
Specificity: When it's actually no, how often does it predict no? default_ind = 0, {TN/actual NO}
Cross Validation Score: Cross Validation is a technique which involves reserving a particular sample of a dataset on which you do not train the model. Later, you test your model on this sample before finalizing it. Do this for k folds and take mean of accuracy scores of the k fold models.
F1 Score: This is a weighted average of the true positive rate (recall) and precision.
ROC Curve: This is a commonly used graph that summarizes the performance of a classifier over all possible thresholds. It is generated by plotting the True Positive Rate (y-axis) against the False Positive Rate (x-axis) as you vary the threshold for assigning observations to a given class.
Above information was taken and more details can be found at https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

Step 5.1 - Prepare Train & Test Dataset
The data should be divided into train and test data. We will use train_test_split feature to divide the data and we will be using 70-30 ratio
"""
# Import the train_test_split method
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# Split data into train and test sets as well as for validation and testing
X_train, X_val, y_train, y_val = train_test_split(final_emp_df, attrition_target,
                                                  test_size= 0.30, random_state=42);

print("Stratified Sampling: ", len(X_train), "train set +", len(X_val), "validation set")

# Stratified Splitting
#split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
#for train_index, test_index in split.split(emp_data_proc, emp_data_proc['Gender']):
#    strat_train_set = emp_data_proc.loc[train_index]
#    strat_test_set = emp_data_proc.loc[test_index]





















from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
def gen_model_performance(actual_target, pred_target):
    model_conf_matrix = confusion_matrix(actual_target, pred_target)
    model_roc_score = roc_auc_score(actual_target, pred_target)
    model_accuracy = accuracy_score(actual_target, pred_target) * 100.0
    
    TP = model_conf_matrix[1][1]; TN = model_conf_matrix[0][0]; 
    FP = model_conf_matrix[0][1]; FN = model_conf_matrix[1][0];
    sensitivity = TP / float(TP + FN) * 100.0; specificity = TN / float(TN + FP) * 100.0;
    precision = TP / float(TP + FP) * 100.0;
    
    return sensitivity, specificity, model_accuracy, precision, model_roc_score
def evaluate_model_score(X, y, scoring='accuracy'):
    
    logreg_model = LogisticRegression(random_state=0)
    logreg_cv_model = LogisticRegressionCV()
    rfc_model = RandomForestClassifier()
    extrees_model = ExtraTreesClassifier()
    gboost_model = GradientBoostingClassifier()
    dt_model = DecisionTreeClassifier()
    aboost_model = AdaBoostClassifier()
    gnb_model = GaussianNB()

    models = [logreg_model, logreg_cv_model, dt_model, rfc_model, 
              extrees_model, gboost_model, aboost_model, gnb_model]
    
    model_results = pd.DataFrame(columns = ["Model", "Accuracy", "Precision", "CV Score",
                                            "Sensitivity","Specificity","ROC Score"])
    
    for model in models:
        model.fit(X, y,)
        y_pred = model.predict(X)
        score = cross_val_score(model, X, y, cv=5, scoring=scoring)
        
        sensitivity, specificity, accuracy, precision, roc_score = gen_model_performance(y, y_pred)
    
        scores = cross_val_score(model, X, y, cv=5)
    
        model_results = model_results.append({"Model": model.__class__.__name__,
                              "Accuracy": accuracy, "Precision": precision,
                              "CV Score": scores.mean()*100.0,
                              "Sensitivity": sensitivity, "Specificity": specificity,
                              "ROC Score": roc_score}, ignore_index=True)
    return model_results
 """

In [ ]:
fd1.info()

In [ ]:
llist=[fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  i.CURRENT_RATING.fillna(i.LAST_RATING,inplace=True)
  i.CURRENT_RATING.fillna(i.ALL_PREVIOUS_RAINGS.str[-1],inplace=True)

In [ ]:
for i in range(fd1[fd1.CURRENT_RATING.isnull()].ALL_PREVIOUS_RAING.shape[0]):
  a=fd1[fd1.CURRENT_RATING.isnull()].ALL_PREVIOUS_RAING[i]
  print(a[-1])

In [ ]:
llist=[fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  i.PRIMARY_SKILL.fillna(i.SECONDARY_SKILL,inplace=True)

In [ ]:
'''s='20170601'
date=pd.to_datetime(s)
date'''

In [ ]:
for i in list_df:
  i.replace(' ', np.nan, inplace=True)

In [ ]:
da1=pd.to_datetime('20160301')
da2=pd.to_datetime('20160601')
fd1['PROMOTION_FLAG']=fd1.apply(lambda fd1 : 1 if fd1.LAST_PROMOTION_DATE.date()>=da1.date() and fd1.LAST_PROMOTION_DATE.date()<=da2.date()  else 0,axis=1)
da1=pd.to_datetime('20160601')
da2=pd.to_datetime('20160901')
fd2['PROMOTION_FLAG']=fd2.apply(lambda fd2 : 1 if fd2.LAST_PROMOTION_DATE.date()>da1.date() and fd2.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20160901')
da2=pd.to_datetime('20161201')
fd3['PROMOTION_FLAG']=fd3.apply(lambda fd3 : 1 if fd3.LAST_PROMOTION_DATE.date()>da1.date() and fd3.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20161201')
da2=pd.to_datetime('20170301')
fd4['PROMOTION_FLAG']=fd4.apply(lambda fd4 : 1 if fd4.LAST_PROMOTION_DATE.date()>da1.date() and fd4.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20170301')
da2=pd.to_datetime('20170601')
fd5['PROMOTION_FLAG']=fd5.apply(lambda fd5 : 1 if fd5.LAST_PROMOTION_DATE.date()>da1.date() and fd5.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20170601')
da2=pd.to_datetime('20170901')
fd6['PROMOTION_FLAG']=fd6.apply(lambda fd6 : 1 if fd6.LAST_PROMOTION_DATE.date()>da1.date() and fd6.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20170901')
da2=pd.to_datetime('20171201')
fd7['PROMOTION_FLAG']=fd7.apply(lambda fd7 : 1 if fd7.LAST_PROMOTION_DATE.date()>da1.date() and fd7.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)
da1=pd.to_datetime('20171201')
da2=pd.to_datetime('20180301')
fd8['PROMOTION_FLAG']=fd8.apply(lambda fd8 : 1 if fd8.LAST_PROMOTION_DATE.date()>da1.date() and fd8.LAST_PROMOTION_DATE.date()<da2.date() else 0,axis=1)

In [ ]:
da=datetime(2016,1,1)
# fd1.LAST_PROMOTION_DATE.dt.date<da.date()
# fd1.LAST_PROMOTION_DATE.dt.date
# # fd1.LAST_PROMOTION_DATE.dtypespd
# pd.to_datetime(da)
# # da.date()
# np.datetime64(da)
da.date().month

In [ ]:
fd1.LAST_PROMOTION_DATE.dt.date